10.1 GroupBy Mechanics

In [4]:
import pandas as pd
import numpy as np


df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df
grouped = df['data1'].groupby(df['key1'])
grouped
grouped.mean()


key1
a    0.240843
b    0.104298
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.549970
      two    -0.377410
b     one     0.488064
      two    -0.279469
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,0.549970,-0.377410
b,0.488064,-0.279469


In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005   -0.377410
            2006    0.488064
Ohio        2005    0.409349
            2006    0.001773
Name: data1, dtype: float64

In [13]:
df.groupby('key1').mean(numeric_only=True)

,data1,data2
key1,,
a,0.240843,-1.694155
b,0.104298,1.012113


In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.549970 -1.010480
     two  -0.377410 -3.061505
b    one   0.488064  2.202667
     two  -0.279469 -0.178441

In [14]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [15]:
# Iterating Over Groups

for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.098167 -0.835256
1    a  two -0.377410 -3.061505
4    a  one  0.001773 -1.185705
b
  key1 key2     data1     data2
2    b  one  0.488064  2.202667
3    b  two -0.279469 -0.178441


In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.098167 -0.835256
4    a  one  0.001773 -1.185705
('a', 'two')
  key1 key2    data1     data2
1    a  two -0.37741 -3.061505
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.488064  2.202667
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.279469 -0.178441


In [18]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.488064,2.202667
3,b,two,-0.279469,-0.178441


In [20]:
df.dtypes
for dtype, group in grouped:
    print(dtype)
    print(group)

a
0    1.098167
1   -0.377410
4    0.001773
Name: data1, dtype: float64
b
2    0.488064
3   -0.279469
Name: data1, dtype: float64


In [26]:
# Selecting a Column or Subset of Columns

df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.010480
     two  -3.061505
b    one   2.202667
     two  -0.178441

In [29]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped
s_grouped.mean()

key1  key2
a     one    -1.010480
      two    -3.061505
b     one     2.202667
      two    -0.178441
Name: data2, dtype: float64

In [21]:
# Grouping with Dicts and Series

people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.497707,-1.869093,0.163027,-0.060932,0.719777
Steve,-0.134266,0.350190,-0.149868,-0.251269,-1.036591
Wes,0.116678,NaN,NaN,-0.352340,1.069841
Jim,1.807934,-0.489560,0.562145,-0.135978,-1.046192
Travis,0.131337,-0.264420,0.061621,0.684875,-0.060339


In [23]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}
by_column = people.T.groupby(mapping)
by_column.sum()

,Joe,Steve,Wes,Jim,Travis
blue,0.102095,-0.401137,-0.352340,0.426167,0.746496
red,-0.651609,-0.820667,1.186519,0.272183,-0.193422


In [25]:
map_series = pd.Series(mapping)
map_series
people.T.groupby(map_series).count()

,Joe,Steve,Wes,Jim,Travis
blue,2,2,1,2,2
red,3,3,2,3,3


In [30]:
# Grouping with Functions

people.groupby(len).sum()

,a,b,c,d,e
3,2.422320,-2.358653,0.725171,-0.549249,0.743426
5,-0.134266,0.350190,-0.149868,-0.251269,-1.036591
6,0.131337,-0.264420,0.061621,0.684875,-0.060339


In [31]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.116678 -1.869093  0.163027 -0.352340  0.719777
  two  1.807934 -0.489560  0.562145 -0.135978 -1.046192
5 one -0.134266  0.350190 -0.149868 -0.251269 -1.036591
6 two  0.131337 -0.264420  0.061621  0.684875 -0.060339

In [32]:
# Grouping by Index Levels

columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      0.474790 -1.294720  0.857986 -1.352184 -0.352745
1      1.318117  0.326768  1.865385  0.637871 -1.397068
2     -0.565447 -0.755753  0.111416 -0.305201 -0.310356
3      0.233398 -0.185457  0.769585  0.206446 -0.147800

In [35]:
hier_df.T.groupby(level='city').count()

,0,1,2,3
city,,,,
JP,2,2,2,2
US,3,3,3,3


10.2 Data Aggregation

In [38]:
df
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.878888
b    0.411311
Name: data1, dtype: float64

In [40]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped[grouped.obj.select_dtypes(include='number').columns].agg(peak_to_peak)

,data1,data2
key1,,
a,1.475577,2.226249
b,0.767533,2.381108


In [41]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.240843  0.766288 -0.377410 -0.187818  0.001773  0.549970   
b      2.0  0.104298  0.542728 -0.279469 -0.087586  0.104298  0.296181   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.098167   3.0 -1.694155  1.197054 -3.061505 -2.123605 -1.185705   
b     0.488064   2.0  1.012113  1.683698 -0.178441  0.416836  1.012113   

                         
          75%       max  
key1                     
a    -1.01048 -0.835256  
b     1.60739  2.202667

In [45]:
# Column-Wise and Multiple Function Application

tips = pd.read_csv('examples/napiwki.csv')
tips
tips['tip_pct'] = tips['napiwek'] / tips['rachunek']
tips[:6]

,rachunek,napiwek,palacz,dzień,pora,wielkość,tip_pct
0,16.99,1.01,nie,niedz.,kolacja,2,0.059447
1,10.34,1.66,nie,niedz.,kolacja,3,0.160542
2,21.01,3.50,nie,niedz.,kolacja,3,0.166587
3,23.68,3.31,nie,niedz.,kolacja,2,0.139780
4,24.59,3.61,nie,niedz.,kolacja,4,0.146808
5,25.29,4.71,nie,niedz.,kolacja,4,0.186240


In [46]:
grouped = tips.groupby(['dzień', 'palacz'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

dzień   palacz
czw.    nie       0.160298
        tak       0.163863
niedz.  nie       0.160113
        tak       0.187250
pt.     nie       0.151650
        tak       0.174783
sob.    nie       0.158048
        tak       0.147906
Name: tip_pct, dtype: float64

In [47]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
dzień  palacz                                  
czw.   nie     0.160298  0.038774      0.193350
       tak     0.163863  0.039389      0.151240
niedz. nie     0.160113  0.042347      0.193226
       tak     0.187250  0.154134      0.644685
pt.    nie     0.151650  0.028123      0.067349
       tak     0.174783  0.051293      0.159925
sob.   nie     0.158048  0.039767      0.235193
       tak     0.147906  0.061375      0.290095

In [50]:
grouped_pct.agg([('foo', 'mean'), ('bar', 'std')])

foo       bar
dzień  palacz                    
czw.   nie     0.160298  0.038774
       tak     0.163863  0.039389
niedz. nie     0.160113  0.042347
       tak     0.187250  0.154134
pt.    nie     0.151650  0.028123
       tak     0.174783  0.051293
sob.   nie     0.158048  0.039767
       tak     0.147906  0.061375

In [53]:
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'rachunek']].agg(functions)
result

tip_pct                     rachunek                  
                count      mean       max    count       mean    max
dzień  palacz                                                       
czw.   nie         45  0.160298  0.266312       45  17.113111  41.19
       tak         17  0.163863  0.241255       17  19.190588  43.11
niedz. nie         57  0.160113  0.252672       57  20.506667  48.17
       tak         19  0.187250  0.710345       19  24.120000  45.35
pt.    nie          4  0.151650  0.187735        4  18.420000  22.75
       tak         15  0.174783  0.263480       15  16.813333  40.17
sob.   nie         45  0.158048  0.291990       45  19.661778  48.33
       tak         42  0.147906  0.325733       42  21.276667  50.81

In [54]:
result['tip_pct']

count      mean       max
dzień  palacz                           
czw.   nie        45  0.160298  0.266312
       tak        17  0.163863  0.241255
niedz. nie        57  0.160113  0.252672
       tak        19  0.187250  0.710345
pt.    nie         4  0.151650  0.187735
       tak        15  0.174783  0.263480
sob.   nie        45  0.158048  0.291990
       tak        42  0.147906  0.325733

In [56]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', 'var')]
grouped[['tip_pct', 'rachunek']].agg(ftuples)

tip_pct                rachunek            
              Durchschnitt Abweichung Durchschnitt  Abweichung
dzień  palacz                                                 
czw.   nie        0.160298   0.001503    17.113111   59.625081
       tak        0.163863   0.001551    19.190588   69.808518
niedz. nie        0.160113   0.001793    20.506667   66.099980
       tak        0.187250   0.023757    24.120000  109.046044
pt.    nie        0.151650   0.000791    18.420000   25.596333
       tak        0.174783   0.002631    16.813333   82.562438
sob.   nie        0.158048   0.001581    19.661778   79.908965
       tak        0.147906   0.003767    21.276667  101.387535

In [60]:
grouped.agg({'napiwek': 'max', 'wielkość': 'sum'})

napiwek  wielkość
dzień  palacz                   
czw.   nie        6.70       112
       tak        5.00        40
niedz. nie        6.00       167
       tak        6.50        49
pt.    nie        3.50         9
       tak        4.73        31
sob.   nie        9.00       115
       tak       10.00       104

In [61]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'wielkość': 'sum'})

tip_pct                               wielkość
                    min       max      mean       std      sum
dzień  palacz                                                 
czw.   nie     0.072961  0.266312  0.160298  0.038774      112
       tak     0.090014  0.241255  0.163863  0.039389       40
niedz. nie     0.059447  0.252672  0.160113  0.042347      167
       tak     0.065660  0.710345  0.187250  0.154134       49
pt.    nie     0.120385  0.187735  0.151650  0.028123        9
       tak     0.103555  0.263480  0.174783  0.051293       31
sob.   nie     0.056797  0.291990  0.158048  0.039767      115
       tak     0.035638  0.325733  0.147906  0.061375      104

In [63]:
# Returning Aggregated Data Without Row Indexes

tips.groupby(['dzień', 'palacz'], as_index=False).mean(numeric_only=True)

,dzień,palacz,rachunek,napiwek,wielkość,tip_pct
0,czw.,nie,17.113111,2.673778,2.488889,0.160298
1,czw.,tak,19.190588,3.030000,2.352941,0.163863
2,niedz.,nie,20.506667,3.167895,2.929825,0.160113
3,niedz.,tak,24.120000,3.516842,2.578947,0.187250
4,pt.,nie,18.420000,2.812500,2.250000,0.151650
5,pt.,tak,16.813333,2.714000,2.066667,0.174783
6,sob.,nie,19.661778,3.102889,2.555556,0.158048
7,sob.,tak,21.276667,2.875476,2.476190,0.147906
